---
# CitiBike Analysis Notebook
---

In [1]:
import numpy as np
import pandas as pd
import json
import urllib.request
import veroviz as vrv
import os

--- 
## 1. Import Station Info (from .json)
---

In [2]:
# Grab JSON data:
with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_information.json") as url:
    station_info_data = json.loads(url.read().decode())

In [14]:
# Convert the JSON data into a Pandas DF:
station_info_df = pd.DataFrame(station_info_data['data']['stations'])
station_info_df = station_info_df.drop('eightd_station_services', axis = 1)
station_info_df.head(3)

,station_id,external_id,name,short_name,lat,lon,region_id,rental_methods,capacity,rental_url,electric_bike_surcharge_waiver,eightd_has_key_dispenser,has_kiosk
0,304,66db6da2-0aca-11e7-82f6-3863bb44ef7c,Broadway & Battery Pl,4962.01,40.704633,-74.013617,71,"[KEY, CREDITCARD]",33,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,True,True
1,359,66dbc982-0aca-11e7-82f6-3863bb44ef7c,E 47 St & Park Ave,6584.12,40.755103,-73.974987,71,"[KEY, CREDITCARD]",64,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,False,True
2,367,66dbcdfc-0aca-11e7-82f6-3863bb44ef7c,E 53 St & Lexington Ave,6617.09,40.758281,-73.970694,71,"[KEY, CREDITCARD]",34,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,False,False,True


---
## 2.  Import Station Status (from .json)
---

In [18]:
# Grab JSON data:
with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_status.json") as url:
    station_status_data = json.loads(url.read().decode())

In [19]:
# Convert the data into a Pandas DF, and add "color" columns to be used for nodes:
station_status_df = pd.DataFrame(station_status_data['data']['stations'])
station_status_df = station_status_df.drop('eightd_active_station_services', axis = 1)
station_status_df['colors'] = None
station_status_df['CESIUMcolors'] = None

In [20]:
# Populate "color" columns depending on the available docks and bikes at a station
pd.set_option('mode.chained_assignment', None)
for i in station_status_df.index:
    if station_status_df['num_docks_available'][i] >= 6 and station_status_df['num_bikes_available'][i] >= 6:
        station_status_df['colors'][i] = 'green'
        station_status_df['CESIUMcolors'][i] = 'Cesium.Color.GREEN'
    elif station_status_df['num_docks_available'][i] <= 5:
        station_status_df['colors'][i] = 'lightred'
        station_status_df['CESIUMcolors'][i] = 'Cesium.Color.YELLOW'
    elif station_status_df['num_bikes_available'][i] <= 5:
        station_status_df['colors'][i] = 'red'
        station_status_df['CESIUMcolors'][i] = 'Cesium.Color.RED'
station_status_df.head(3)

,station_id,num_bikes_available,num_ebikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,eightd_has_available_keys,colors,CESIUMcolors
0,304,3,1,1,29,0,1,1,1,1582812671,True,red,Cesium.Color.RED
1,359,13,2,2,49,0,1,1,1,1582812678,False,green,Cesium.Color.GREEN
2,367,4,0,0,30,0,1,1,1,1582812459,False,red,Cesium.Color.RED


--- 
## 3.  Import Trip Data (from .csv)
---

In [21]:
# Trip data
bike_trips_df = pd.read_csv('citibike_tripdata.csv')

# PULL OUT ONE BIKE TO ANALYZE FOR THE REMAINDER OF THIS NOTEBOOK
bike_trips_df2 = bike_trips_df[bike_trips_df['bikeid'] == 14530].copy()
bike_trips_df2['cesiumColor'] = 'Cesium.Color.BLUE'
bike_trips_df2['leafletColor'] = 'blue'
bike_trips_df2.head(3)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,cesiumColor,leafletColor
26822,548,2020-01-02 09:25:11.9390,2020-01-02 09:34:20.3600,261,Johnson St & Gold St,40.694749,-73.983625,2000,Front St & Washington St,40.702551,-73.989402,14530,Subscriber,1987,1,Cesium.Color.BLUE,blue
50614,1061,2020-01-02 18:16:45.1630,2020-01-02 18:34:26.5780,2000,Front St & Washington St,40.702551,-73.989402,3414,Bergen St & Flatbush Ave,40.680945,-73.975673,14530,Subscriber,1992,1,Cesium.Color.BLUE,blue
134556,400,2020-01-05 14:27:01.9750,2020-01-05 14:33:42.1210,3414,Bergen St & Flatbush Ave,40.680945,-73.975673,3486,Schermerhorn St & Bond St,40.688417,-73.984517,14530,Customer,1969,0,Cesium.Color.BLUE,blue


--- 
## 4. Create VeRoViz "nodes" Dataframes (one for all nodes and one for the nodes that bike 14530 visited)
---

In [22]:
# Nodes DF for all stations
nodes = vrv.initDataframe('nodes')
nodes['id'] = station_info_df['station_id'].values
nodes[['id', 'lat', 'lon', 'nodeName']] = station_info_df[['station_id', 'lat', 'lon', 'name']].values
nodes[['leafletIconText', 'cesiumIconText']] = station_info_df[['name', 'station_id']].values
nodes['leafletColor'] = station_status_df['colors'].values
nodes['cesiumColor'] = station_status_df['CESIUMcolors'].values
nodes.loc[:,'altMeters'] = 0
nodes.loc[:,['nodeType', 'leafletIconPrefix', 'leafletIconType']] = [
             'Citibike Station', 'fa', 'bicycle']
nodes.loc[:,'cesiumIconType'] = 'pin'
nodes.head(3)

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,304,40.7046,-74.0136,0,Broadway & Battery Pl,Citibike Station,fa,bicycle,red,Broadway & Battery Pl,pin,Cesium.Color.RED,304
1,359,40.7551,-73.975,0,E 47 St & Park Ave,Citibike Station,fa,bicycle,green,E 47 St & Park Ave,pin,Cesium.Color.GREEN,359
2,367,40.7583,-73.9707,0,E 53 St & Lexington Ave,Citibike Station,fa,bicycle,red,E 53 St & Lexington Ave,pin,Cesium.Color.RED,367


In [23]:
# This block extracts the station attributes for the nodes that bike 14530 visited

# Extract attributes (station name, station id, latitude, and longitude)
_stations_name = bike_trips_df2['start station name'].tolist()
_stations_name.extend(bike_trips_df2['end station name'].tolist())
_stations_lat = bike_trips_df2['start station latitude'].tolist()
_stations_lat.extend(bike_trips_df2['end station latitude'].tolist())
_stations_lon = bike_trips_df2['start station longitude'].tolist()
_stations_lon.extend(bike_trips_df2['end station longitude'].tolist())
_stations_id = bike_trips_df2['start station id'].tolist()
_stations_id.extend(bike_trips_df2['end station id'].tolist())

# Create a DF with the above attributes. This will be used to populate a nodes DF for the chosen bike
_stations_df = pd.DataFrame(columns = ['station_id', 'lat', 'lon', 'name'])
_stations_df['station_id'] = _stations_id
_stations_df['lat'] = _stations_lat
_stations_df['lon'] = _stations_lon
_stations_df['name'] = _stations_name
_stations_df = _stations_df.drop_duplicates(subset='station_id')

# Merge this DF with 'station_status' DF to get the proper colors for the nodes that the chosen bike visited
_stations_df['station_id'] = _stations_df['station_id'].astype(int)
station_status_df['station_id'] = station_status_df['station_id'].astype(int)
_stations_df = _stations_df.merge(station_status_df, left_on = 'station_id', right_on = 'station_id')
_stations_df.head(3)

,station_id,lat,lon,name,num_bikes_available,num_ebikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,eightd_has_available_keys,colors,CESIUMcolors
0,261,40.694749,-73.983625,Johnson St & Gold St,5,0,0,22,0,1,1,1,1582811951,False,red,Cesium.Color.RED
1,2000,40.702551,-73.989402,Front St & Washington St,25,1,0,5,0,1,1,1,1582812246,False,lightred,Cesium.Color.YELLOW
2,3414,40.680945,-73.975673,Bergen St & Flatbush Ave,32,0,0,1,0,1,1,1,1582810723,False,lightred,Cesium.Color.YELLOW


In [24]:
# New nodes DF -- only for bike 14530
nodes2 = vrv.initDataframe('nodes')

#Populate nodes DF with '_stations_df'
nodes2['id'] = _stations_df['station_id'].values
nodes2[['id', 'lat', 'lon', 'nodeName']] = _stations_df[['station_id', 'lat', 'lon', 'name']].values
nodes2[['leafletIconText', 'cesiumIconText']] = _stations_df[['name', 'station_id']].values
nodes2['leafletColor'] = _stations_df['colors'].values
nodes2['cesiumColor'] = _stations_df['CESIUMcolors'].values
nodes2.loc[:,'altMeters'] = 0
nodes2.loc[:,['nodeType', 'leafletIconPrefix', 'leafletIconType']] = [
             'Citibike Station', 'fa', 'bicycle']
nodes2.loc[:,'cesiumIconType'] = 'pin'
nodes2.head(3)

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,261,40.694749,-73.983625,0,Johnson St & Gold St,Citibike Station,fa,bicycle,red,Johnson St & Gold St,pin,Cesium.Color.RED,261
1,2000,40.702551,-73.989402,0,Front St & Washington St,Citibike Station,fa,bicycle,lightred,Front St & Washington St,pin,Cesium.Color.YELLOW,2000
2,3414,40.680945,-73.975673,0,Bergen St & Flatbush Ave,Citibike Station,fa,bicycle,lightred,Bergen St & Flatbush Ave,pin,Cesium.Color.YELLOW,3414


In [17]:
# Color codes FOR LEAFLET:
#    green --> bikes and docks are available
#    red --> less than 6 bikes available
#    lightred --> less than 6 docks available

# Color codes FOR CESIUM:
#    green --> bikes and docks are available
#    red --> less than 6 bikes available
#    yellow --> less than 6 docks available

In [28]:
# Show all of the stations on a Leaflet map:
vrv.createLeaflet(nodes=nodes)

In [29]:
# Show statins only for bike 14530:
vrv.createLeaflet(nodes=nodes2)

--- 
## 5. Create VeRoViz "assignments" Dataframe
---

In [30]:
# Produce a "timestamp" (days HH:MM:SS.ms) showing the time since the first observed 'starttime':
bike_trips_df2['timeAfterStart'] = pd.to_datetime(bike_trips_df2['starttime']) - \
                                  pd.to_datetime(min(bike_trips_df2['starttime']))

# Convert this to a decimal number of seconds:
bike_trips_df2['timeAfterStart'] = bike_trips_df2['timeAfterStart'].dt.total_seconds().astype(int)
bike_trips_df2['timeAfterStart'].head(3)

26822          0
50614      31893
134556    277310
Name: timeAfterStart, dtype: int64

In [31]:
# Assignments DF:
assignments = vrv.initDataframe('assignments')

# Copy over the static values:
assignments['objectID'] = bike_trips_df2['bikeid'].values
assignments[['startLat', 'startLon', 'endLat', 'endLon']] = bike_trips_df2[['start station latitude', 
                                                                          'start station longitude',
                                                                          'end station latitude',
                                                                          'end station longitude']].values

# Copy new calculated column:
assignments['startTimeSec'] = bike_trips_df2['timeAfterStart'].values

# Use the calculated column and tripduration to get the end time:
assignments['endTimeSec'] = (bike_trips_df2['timeAfterStart'] + bike_trips_df2['tripduration']).values

# Fill in the rest of our assignments DF with hard-coded values:
assignments.loc[:,['modelFile', 'modelScale', 'modelMinPxSize', 'startAltMeters', 'endAltMeters', 
                   'leafletWeight', 'leafletStyle', 'leafletOpacity', 'useArrows',
                   'cesiumWeight', 'cesiumStyle', 'cesiumOpacity']] = \
                  ['veroviz/models/car_green.gltf', 100, 45, 0, 0, 
                   2, 'solid', 0.8, False, 
                   2, 'solid', 0.7]
assignments['leafletColor'] = bike_trips_df2['leafletColor'].values
assignments['cesiumColor'] = bike_trips_df2['cesiumColor'].values
assignments['odID'] = bike_trips_df2.index
assignments.head(3)

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,26822,14530,veroviz/models/car_green.gltf,100,45,0,40.694749,-73.983625,0,548,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1,50614,14530,veroviz/models/car_green.gltf,100,45,31893,40.702551,-73.989402,0,32954,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
2,134556,14530,veroviz/models/car_green.gltf,100,45,277310,40.680945,-73.975673,0,277710,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7


In [34]:
# Create a new DF containing the repositioning of the chosen bike **USING FOR LOOP**
reposition_df = pd.DataFrame(columns = ['objectID', 'endLat', 'endLon', 'endTimeSec',\
                                        'startLat', 'startLon', 'startTimeSec', 'leafletColor','cesiumColor'])
reposition_endLat = []
reposition_endLon = []
reposition_startLat = []
reposition_startLon = []
reposition_endtime = []
reposition_starttime = []

for i in range(len(assignments)-1):
    if assignments['endLat'][i] != assignments['startLat'][i+1]:
        reposition_endLat.append(assignments['endLat'][i])
        reposition_endLon.append(assignments['endLon'][i])
        reposition_endtime.append(assignments['endTimeSec'][i])
        reposition_startLat.append(assignments['startLat'][i+1])
        reposition_startLon.append(assignments['startLon'][i+1])
        reposition_starttime.append(assignments['startTimeSec'][i+1])
    else:
        pass
    
reposition_df['endLat'] = reposition_endLat
reposition_df['endLon'] = reposition_endLon
reposition_df['endTimeSec'] = reposition_endtime
reposition_df['startLat'] = reposition_startLat
reposition_df['startLon'] = reposition_startLon
reposition_df['startTimeSec'] = reposition_starttime
reposition_df['objectID'] = 14530
reposition_df['leafletColor'] = 'red' 
reposition_df['cesiumColor'] = 'Cesium.Color.RED'
reposition_df

,objectID,endLat,endLon,endTimeSec,startLat,startLon,startTimeSec,leafletColor,cesiumColor
0,14530,40.689622,-73.983043,390865,40.678612,-73.990373,446280,red,Cesium.Color.RED
1,14530,40.733320,-73.995101,1211025,40.729538,-73.984267,1218929,red,Cesium.Color.RED
2,14530,40.734814,-73.992085,1219217,40.732219,-73.981656,1295415,red,Cesium.Color.RED
3,14530,40.745497,-74.001971,1324120,40.766741,-73.979069,1488342,red,Cesium.Color.RED


In [33]:
# Create a new DF containing the repositioning of the chosen bike **NOT USING FOR LOOP**
reposition_df2 = assignments.copy()
assignments_shifted = reposition_df2.iloc[1:].reset_index(drop = True)
assignments_shifted = assignments_shifted.append(pd.Series(), ignore_index = True)
_bike_start = assignments_shifted['startLat'].tolist()
_bike_end = assignments_shifted['startLon'].tolist()
reposition_df2['start lat for next trip'] = _bike_start
reposition_df2['start lon for next trip'] = _bike_end
reposition_df2['repositioning'] = 0
reposition_df2.loc[(reposition_df2['endLat'] != \
                    reposition_df2['start lat for next trip']), 'repositioning'] = 1
reposition_df2.iloc[-1, reposition_df2.columns.get_loc('repositioning')] = 0
reposition_df2 = reposition_df2[reposition_df2['repositioning'] == 1]
reposition_df2['leafletColor'] = 'red' 
reposition_df2['cesiumColor'] = 'Cesium.Color.RED'
reposition_df2

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity,start lat for next trip,start lon for next trip,repositioning
8,190020,14530,veroviz/models/car_green.gltf,100,45,390424,40.680213,-73.984327,0,390865,...,solid,0.8,False,Cesium.Color.RED,2,solid,0.7,40.678612,-73.990373,1
44,641708,14530,veroviz/models/car_green.gltf,100,45,1210613,40.731724,-74.006744,0,1211025,...,solid,0.8,False,Cesium.Color.RED,2,solid,0.7,40.729538,-73.984267,1
45,646253,14530,veroviz/models/car_green.gltf,100,45,1218929,40.729538,-73.984267,0,1219217,...,solid,0.8,False,Cesium.Color.RED,2,solid,0.7,40.732219,-73.981656,1
49,696639,14530,veroviz/models/car_green.gltf,100,45,1323703,40.739355,-73.999318,0,1324120,...,solid,0.8,False,Cesium.Color.RED,2,solid,0.7,40.766741,-73.979069,1


In [36]:
# New assignment DF for repositioning
assignments2 = vrv.initDataframe('assignments')

# Copy over the static values:
assignments2['objectID'] = reposition_df['objectID'].values
assignments2[['startLat', 'startLon', 'endLat', 'endLon']] = reposition_df[['endLat', 
                                                                          'endLon',
                                                                          'startLat',
                                                                          'startLon']].values
assignments2['startTimeSec'] = reposition_df['startTimeSec'].values
assignments2['endTimeSec'] = reposition_df['endTimeSec'].values
assignments2.loc[:,['modelFile', 'modelScale', 'modelMinPxSize', 'startAltMeters', 'endAltMeters', 
                   'leafletColor', 'leafletWeight', 'leafletStyle', 'leafletOpacity', 'useArrows',
                   'cesiumColor', 'cesiumWeight', 'cesiumStyle', 'cesiumOpacity']] = \
                  ['veroviz/models/car_green.gltf', 100, 45, 0, 0, 
                   'red', 2, 'solid', 0.8, False, 
                   'Cesium.Color.RED', 2, 'solid', 0.7]
assignments2.loc[:,'odID'] = list(range(0, len(assignments2)))
assignments2.head(3)
assignments = assignments.append(assignments2.copy())
assignments = assignments.sort_values('endTimeSec', ascending = True)
assignments = assignments.reset_index(drop = True)
assignments.head(3)

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,26822,14530,veroviz/models/car_green.gltf,100,45,0,40.694749,-73.983625,0,548,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1,50614,14530,veroviz/models/car_green.gltf,100,45,31893,40.702551,-73.989402,0,32954,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
2,134556,14530,veroviz/models/car_green.gltf,100,45,277310,40.680945,-73.975673,0,277710,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7


In [41]:
# Add static assignments to ensure that bike does not dissapear when docked at a station
assignments_docked = assignments.loc[assignments['leafletColor'] == 'blue'].copy()
for i in range(len(assignments_docked)-1):
    assignments_docked = vrv.addStaticAssignment(
            initAssignments = assignments_docked,
            odID            = assignments_docked.iloc[i]['odID'],
            objectID        = assignments_docked.iloc[i]['objectID'],
            modelFile       = 'veroviz/models/car_green.gltf',
            modelScale      = 100,
            modelMinPxSize  = 45,
            loc             = [assignments_docked.iloc[i]['endLat'] , assignments_docked.iloc[i]['endLon'],\
                               assignments_docked.iloc[i]['endAltMeters']],
            startTimeSec    = assignments_docked.iloc[i]['endTimeSec'],
            endTimeSec      = assignments_docked.iloc[i+1]['startTimeSec'])           
assignments3 = assignments_docked.copy()
assignments3 = assignments3.append(assignments2.copy())
assignments3 = assignments3.reset_index(drop = True)
assignments3.head(3)

,odID,objectID,modelFile,modelScale,modelMinPxSize,startTimeSec,startLat,startLon,startAltMeters,endTimeSec,...,endAltMeters,leafletColor,leafletWeight,leafletStyle,leafletOpacity,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity
0,26822,14530,/veroviz/models/car_green.gltf,100,45,0,40.694749,-73.983625,0,548,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
1,50614,14530,/veroviz/models/car_green.gltf,100,45,31893,40.702551,-73.989402,0,32954,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7
2,134556,14530,/veroviz/models/car_green.gltf,100,45,277310,40.680945,-73.975673,0,277710,...,0,blue,2,solid,0.8,False,Cesium.Color.BLUE,2,solid,0.7


--- 
## 6. Create a Leaflet map 
---

In [42]:
# Show all of the arcs for the chosen bike:
vrv.createLeaflet(arcs=assignments3)

In [43]:
# Show all of the ride arcs for the chosen bike:
vrv.createLeaflet(arcs=assignments3[assignments3['leafletColor'] != 'red'])

In [44]:
# Show all of the repositioning arcs for the chosen bike:
vrv.createLeaflet(arcs=assignments3[assignments3['leafletColor'] == 'red'])

In [45]:
# Done (whole ticket has been completed)

# We'll want to identify when repositioning of bikes by the CitiBike staff occurs.

# Done --> OPTION 1: Use a "for" loop 
# Done --> OPTION 2: Don't use a "for" loop
#          New DF with shifted rows. Check rows and create a flag column for when end lat/lon != start lat/lon
#          Done -- >One large assignments DF (leaflet with just bike trips, repositioning, and both on same map)

# Done --> Draw on a (separate) Leaflet map and a (separate?) Cesium movie the repositioning of bikes
# Done --> Use vrv.addStaticAssignment() to display stationary bikes.

--- 
## 7. Create Cesium movies for one bike
---

In [46]:
# startDate: Format is "YYYY-MM-DD"
startDate = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%Y-%m-%d')

# startTime: Format is "HH:MM:SS"
startTime = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%H:%M:%S')

vrv.createCesium(
    assignments = assignments3,
    nodes       = nodes2,
    startDate   = startDate,
    startTime   = startTime,
    cesiumDir   = os.environ['CESIUMDIR'],
    problemDir  = 'IE_670/static_assignments_2.27.20')

Message: File selector was written to /Users/kylehunt/development/Cesium/IE_670/static_assignments_2.27.20/;IE_670;static_assignments_2.27.20.vrv ...
Message: Configs were written to /Users/kylehunt/development/Cesium/IE_670/static_assignments_2.27.20/config.js ...
Message: Nodes were written to /Users/kylehunt/development/Cesium/IE_670/static_assignments_2.27.20/displayNodes.js ...
Message: Assignments (.js) were written to /Users/kylehunt/development/Cesium/IE_670/static_assignments_2.27.20/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/kylehunt/development/Cesium/IE_670/static_assignments_2.27.20/routes.czml ...


In [36]:
# startDate: Format is "YYYY-MM-DD"
startDate = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%Y-%m-%d')

# startTime: Format is "HH:MM:SS"
startTime = pd.to_datetime(min(bike_trips_df['starttime'])).strftime('%H:%M:%S')

vrv.createCesium(
    assignments = assignments3[assignments3['leafletColor'] == 'red'],
    nodes       = nodes2,
    startDate   = startDate,
    startTime   = startTime,
    cesiumDir   = os.environ['CESIUMDIR'],
    problemDir  = 'IE_670/citibike_repositioning')

Message: File selector was written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/;IE_670;citibike_repositioning.vrv ...
Message: Configs were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/config.js ...
Message: Nodes were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/displayNodes.js ...
Message: Assignments (.js) were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/displayPaths.js ...
Message: Assignments (.czml) were written to /Users/kylehunt/development/Cesium/IE_670/citibike_repositioning/routes.czml ...


In [37]:
# DONE

# The Cesium movie is cluttered with all of our station markers.
# It would be better to only include the markers that are actually relevant 
# to our given bike.

# Fortunately, our bike trips df contains the station IDs.
# We just need to get a list of unique IDs, and then 
# pass to createCesium only the subset of nodes corresponding to these IDs.